In [5]:
# Function to generate comprehensive sales report
from datetime import datetime

def format_currency(value):
    return f"₹{value:,.2f}"

def generate_sales_report(transactions,enriched_data, output_file='output/sales_report.txt'):
    """
    Generates a comprehensive formatted sales analytics text report.
    """

    # ---------------- HEADER ----------------
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    total_records = len(transactions)

    # ---------------- OVERALL SUMMARY ----------------
    total_revenue = sum(t['Quantity'] * t['UnitPrice'] for t in transactions)
    total_transactions = len(transactions)
    avg_order_value = total_revenue / total_transactions if total_transactions else 0
    dates = [t['Date'] for t in transactions]
    date_range = f"{min(dates)} to {max(dates)}" if dates else "N/A"

    # ---------------- REGION PERFORMANCE ----------------
    region_stats = {}
    for t in transactions:
        region = t.get('Region', 'Unknown') or 'Unknown'
        revenue = t['Quantity'] * t['UnitPrice']
        region_stats.setdefault(region, {'sales': 0, 'count': 0})
        region_stats[region]['sales'] += revenue
        region_stats[region]['count'] += 1

    sorted_regions = sorted(region_stats.items(), key=lambda x: x[1]['sales'], reverse=True)

    # ---------------- TOP PRODUCTS ----------------
    product_stats = {}
    for t in transactions:
        name = t['ProductName']
        revenue = t['Quantity'] * t['UnitPrice']
        product_stats.setdefault(name, {'qty': 0, 'rev': 0})
        product_stats[name]['qty'] += t['Quantity']
        product_stats[name]['rev'] += revenue

    top_products = sorted(product_stats.items(), key=lambda x: x[1]['rev'], reverse=True)[:5]

    # ---------------- TOP CUSTOMERS ----------------
    customer_stats = {}
    for t in transactions:
        cid = t['CustomerID']
        revenue = t['Quantity'] * t['UnitPrice']
        customer_stats.setdefault(cid, {'spent': 0, 'orders': 0})
        customer_stats[cid]['spent'] += revenue
        customer_stats[cid]['orders'] += 1

    top_customers = sorted(customer_stats.items(), key=lambda x: x[1]['spent'], reverse=True)[:5]

    # ---------------- DAILY SALES ----------------
    daily_stats = {}
    for t in transactions:
        date = t['Date']
        revenue = t['Quantity'] * t['UnitPrice']
        daily_stats.setdefault(date, {'rev': 0, 'tx': 0, 'cust': set()})
        daily_stats[date]['rev'] += revenue
        daily_stats[date]['tx'] += 1
        daily_stats[date]['cust'].add(t['CustomerID'])

    # ---------------- API SUMMARY ----------------
    enriched = [e for e in enriched_data if e.get('API_Match')]
    failed = [e['ProductName'] for e in enriched_data if not e.get('API_Match')]
    success_rate = (len(enriched) / len(enriched_data)) * 100 if enriched_data else 0

    # ---------------- WRITE REPORT ----------------
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("=" * 45 + "\n")
        f.write("       SALES ANALYTICS REPORT\n")
        f.write(f"Generated: {now}\n")
        f.write(f"Records Processed: {total_records}\n")
        f.write("=" * 45 + "\n\n")

        f.write("OVERALL SUMMARY\n")
        f.write("-" * 45 + "\n")
        f.write(f"Total Revenue:        {format_currency(total_revenue)}\n")
        f.write(f"Total Transactions:   {total_transactions}\n")
        f.write(f"Average Order Value:  {format_currency(avg_order_value)}\n")
        f.write(f"Date Range:           {date_range}\n\n")

        f.write("REGION-WISE PERFORMANCE\n")
        f.write("-" * 45 + "\n")
        f.write("Region    Sales           % of Total   Transactions\n")
        for region, stats in sorted_regions:
            percent = (stats['sales'] / total_revenue) * 100 if total_revenue else 0
            f.write(f"{region:<9}{format_currency(stats['sales']):<16}{percent:>8.2f}%{stats['count']:>15}\n")
        f.write("\n")

        f.write("TOP 5 PRODUCTS\n")
        f.write("-" * 45 + "\n")
        f.write("Rank Product               Qty Sold   Revenue\n")
        for i, (name, s) in enumerate(top_products, 1):
            f.write(f"{i:<5}{name:<22}{s['qty']:<10}{format_currency(s['rev'])}\n")
        f.write("\n")

        f.write("TOP 5 CUSTOMERS\n")
        f.write("-" * 45 + "\n")
        f.write("Rank Customer ID   Total Spent    Orders\n")
        for i, (cid, s) in enumerate(top_customers, 1):
            f.write(f"{i:<5}{cid:<14}{format_currency(s['spent']):<14}{s['orders']}\n")
        f.write("\n")

        f.write("DAILY SALES TREND\n")
        f.write("-" * 45 + "\n")
        f.write("Date         Revenue        Transactions  Customers\n")
        for date, s in sorted(daily_stats.items()):
            f.write(f"{date}  {format_currency(s['rev']):<14}{s['tx']:<14}{len(s['cust'])}\n")
        f.write("\n")

        f.write("PRODUCT PERFORMANCE ANALYSIS\n")
        f.write("-" * 45 + "\n")
        best_day = max(daily_stats.items(), key=lambda x: x[1]['rev'])
        f.write(f"Best Selling Day: {best_day[0]} ({format_currency(best_day[1]['rev'])})\n\n")

        f.write("API ENRICHMENT SUMMARY\n")
        f.write("-" * 45 + "\n")
        f.write(f"Total Products Enriched: {len(enriched)}\n")
        f.write(f"Success Rate: {success_rate:.2f}%\n")
        if failed:
            f.write("Products Not Enriched:\n")
            for p in failed:
                f.write(f"- {p}\n")

    print(f"Report successfully generated at {output_file}")
